## Download the Fashion-MNIST dataset

In [ ]:
import os
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)
np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

In [ ]:
!pygmentize fmnist-4.py

## Upload Fashion-MNIST data to S3

In [ ]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
prefix = 'keras2-fashion-mnist'

training_input_path = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')
output_path = 's3://{}/{}/output/'.format(bucket, prefix)
chk_path = 's3://{}/{}/checkpoints/'.format(bucket, prefix)

print(training_input_path)
print(validation_input_path)
print(output_path)
print(chk_path)

## Train with Tensorflow

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='fmnist-4.py',
                          role=role,
                          instance_count=1, 
                          instance_type='ml.p3.2xlarge',
                          framework_version='2.1.0', 
                          py_version='py3',
                          hyperparameters={'epochs': 100},
                          use_spot_instances=True,
                          max_run=3600,                    
                          max_wait=7200)

In [ ]:
objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [
    {'Name': 'val_acc', 'Regex': 'Best val_accuracy: ([0-9\\.]+)'}
]

In [ ]:
from sagemaker.tuner import ContinuousParameter, IntegerParameter, CategoricalParameter

hyperparameter_ranges = {
    'learning-rate': ContinuousParameter(0.1, 0.14),
    'batch-size': IntegerParameter(130, 160),
    'filters1': IntegerParameter(16,256),
    'filters2': IntegerParameter(16,256),
    'dropout-conv': ContinuousParameter(1e-3, 0.5, scaling_type='Logarithmic'),
    #'bn-momentum': ContinuousParameter(0.8, 0.999, scaling_type='ReverseLogarithmic'),
    #'bn-epsilon': ContinuousParameter(1e-6, 1e-2, scaling_type='Logarithmic'),
    #'num-fc': IntegerParameter(1,2),
    #'width-fc': IntegerParameter(16, 128),
    'dropout-fc': ContinuousParameter(1e-3, 0.5, scaling_type='Logarithmic'),
}

In [ ]:
from sagemaker.tuner import HyperparameterTuner

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            objective_type=objective_type,
                            max_jobs=50,
                            max_parallel_jobs=4,
                            early_stopping_type='Auto')

In [ ]:
tuner.fit({'training': training_input_path, 'validation': validation_input_path})

In [ ]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

exp = HyperparameterTuningJobAnalytics(
  hyperparameter_tuning_job_name=tuner.latest_tuning_job.name)

jobs = exp.dataframe()
jobs.drop(['TrainingJobName', 'TrainingEndTime', 'TrainingStartTime', 'TrainingJobStatus'], axis=1, inplace=True)
jobs = jobs.sort_values('FinalObjectiveValue', ascending=0)
jobs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [ ]:
plt.title('Accuracy vs. learning rate')
plt.autoscale()
plt.scatter(jobs['learning-rate'].tolist(), jobs['FinalObjectiveValue'].tolist(), color='red')
plt.show()
plt.title('Accuracy vs. batch size')
plt.autoscale()
plt.scatter(jobs['batch-size'].tolist(), jobs['FinalObjectiveValue'].tolist(), color='green')
plt.show()

In [ ]:
plt.title('Accuracy vs. CNN layer dropout')
plt.autoscale()
plt.scatter(jobs['dropout-conv'].tolist(), jobs['FinalObjectiveValue'].tolist(), color='red')
plt.show()
plt.title('Accuracy vs. FC layer dropout')
plt.autoscale()
plt.scatter(jobs['dropout-fc'].tolist(), jobs['FinalObjectiveValue'].tolist(), color='blue')
plt.show()

In [ ]:
plt.title('Accuracy vs. CNN layer 1 filters')
plt.autoscale()
plt.scatter(jobs['filters1'].tolist(), jobs['FinalObjectiveValue'].tolist(), color='red')
plt.show()
plt.title('Accuracy vs. CNN layer 2 filters')
plt.autoscale()
plt.scatter(jobs['filters2'].tolist(), jobs['FinalObjectiveValue'].tolist(), color='green')
plt.show()

In [ ]:
X=np.array(jobs['learning-rate'])
Y=np.array(jobs['batch-size'])
Z=np.array(jobs['FinalObjectiveValue'])

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(X, Y,Z)
plt.show()

In [ ]:
X=np.array(jobs['filters1'])
Y=np.array(jobs['filters2'])
Z=np.array(jobs['FinalObjectiveValue'])

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(X, Y,Z)
plt.show()

## Deploy

In [ ]:
import time

tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tuner.deploy(
                 initial_instance_count=1, 
                 instance_type='ml.m5.large',
                 endpoint_name=tf_endpoint_name)

## Predict 

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

payload = images.reshape(num_samples, 28, 28, 1)

In [ ]:
response = tf_predictor.predict(payload)
prediction = np.array(response['predictions'])
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

## Clean up

In [ ]:
tf_predictor.delete_endpoint()